In [0]:
HOST="https://gateway.watsonplatform.net/natural-language-understanding/api"
USERNAME="NO"
PASSWORD="NO"

In [0]:
import requests
import json
import csv
import zlib
from requests.auth import HTTPBasicAuth

In [0]:
reviews = requests.get('http://data.insideairbnb.com/canada/on/toronto/2018-08-08/data/reviews.csv.gz')
reviews = zlib.decompress(reviews.content, 16+zlib.MAX_WBITS).decode('utf8')
reviews = list(csv.DictReader(reviews.splitlines()))

In [0]:
reviews = reviews[:50000]

In [5]:
bad_w = {'scary', 'unfriendly', 'sketchy', 'unsafe', 'dangerous'}
good_w = {'safe', 'reassuring', 'nice neighbourhood', 'quiet'}

import random
from collections import defaultdict
from queue import Queue
import threading

#random.seed(100)

sen_positive = defaultdict(float)
sen_negative = defaultdict(float)

p_q = Queue()
n_q = Queue()

w_q = Queue()

def proc():
  sen_positive = defaultdict(float)
  sen_negative = defaultdict(float)
  
  while True:
    review = w_q.get()
    if w_q.qsize() % 500 == 0:
      print(w_q.qsize())
    if review is None:
      p_q.put(sen_positive)
      n_q.put(sen_negative)
      break

    req = {
      "text": review["comments"],
      "features": {
        "sentiment": {
            "targets": list(bad_w) + list(good_w)
        }
      }
    }

    resp = requests.post(HOST + "/v1/analyze?version=2018-03-19", json=req, auth=HTTPBasicAuth(USERNAME, PASSWORD))

    if not resp.ok and resp.json()['error'] != 'cannot locate keyphrase' and resp.json()['error'] != 'invalid request: content is empty':
      print(resp.text)

    resp = resp.json()

    if 'sentiment' in resp:
      listing_id = review['listing_id']
      for target in resp['sentiment']['targets']:
        if target['text'] in good_w:
          sen_positive[listing_id] += target['score']
        else:
          sen_negative[listing_id] -= target['score']

    
    w_q.task_done()

N = 1000

threads = []

for review in reviews:
  w_q.put(review)

for i in range(N):
  t = threading.Thread(target=proc)
  t.start()
  threads.append(t)

w_q.join()

for i in range(N):
  w_q.put(None)
for t in threads:
  t.join()

  #print(review['listing_id'])
  #print(req['text'])
  #print(resp)
  #print()

[(k, p, sen_negative[k]) for k, p in sen_positive.items()]

{
  "language": "de",
  "error": "target(s) not found",
  "code": 400
}
{
  "language": "de",
  "error": "target(s) not found",
  "code": 400
}
{
  "language": "de",
  "error": "target(s) not found",
  "code": 400
}{
  "language": "de",
  "error": "target(s) not found",
  "code": 400
}

{
  "language": "fr",
  "error": "target(s) not found",
  "code": 400
}{
  "language": "fr",
  "error": "target(s) not found",
  "code": 400
}
{
  "language": "fr",
  "error": "target(s) not found",
  "code": 400
}

{
  "language": "nl",
  "error": "unsupported text language: nl",
  "code": 400
}
{
  "language": "de",
  "error": "target(s) not found",
  "code": 400
}{
  "language": "fr",
  "error": "target(s) not found",
  "code": 400
}{
  "language": "zh",
  "error": "unsupported text language: zh",
  "code": 400
}{
  "language": "de",
  "error": "target(s) not found",
  "code": 400
}{
  "language": "fr",
  "error": "target(s) not found",
  "code": 400
}




{
  "language": "fr",
  "error": "target(s) 

[]

In [0]:
p_s = []
n_s = []

try:
  while True:
    p_s.append(p_q.get(False))
except:
  pass

try:
  while True:
    n_s.append(n_q.get(False))
except:
  pass

In [0]:
positive = defaultdict(float)
negative = defaultdict(float)

for pp in p_s:
  for k, v in pp.items():
    positive[k] += v
for nn in n_s:
  for k, v in nn.items():
    negative[k] += v

In [8]:
negative

defaultdict(float,
            {'1006928': 0.383513,
             '1009751': 0.627817,
             '1038085': 0.762861,
             '1080121': 0.784183,
             '1171884': 0.448478,
             '1202734': 2.461411,
             '1256687': 0.779604,
             '1332336': 0.891619,
             '138006': 0.207934,
             '1382277': 0.33169,
             '1523547': 2.483226,
             '1525977': 0.471148,
             '153000': 0.218483,
             '1594005': 0.583845,
             '1638995': 0.409858,
             '1639953': 0.602794,
             '166742': 1.021945,
             '1668445': 0.643629,
             '1747092': 0.550465,
             '1851448': 0.760205,
             '1869711': 2.554076,
             '1943718': 0.426688,
             '1978673': 0.38018,
             '1981668': 0.0,
             '2026887': 0.60356,
             '2154933': 0.561604,
             '229913': 0.503446,
             '2364687': 0.0,
             '23691': 0.93911,
             '2

In [0]:
import json
with open('safety.json', 'w') as f:
  json.dump({k: max(0, p) / (max(0, p) + max(0, negative[k])) * 100 for k, p in positive.items() if max(0, p)}, f)

In [0]:
from google.colab import files
files.download('safety.json')